In [1]:
# device = 'K0303'
# active_power_variable = 'active_power'
path_base_r_string = r'D:\OneDrive - CELSIA S.A E.S.P'

%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings("ignore")

In [2]:
# this cell enables project path relative imports
import sys
from pathlib import Path
path_base = Path(path_base_r_string)
project_path = path_base/'Proyectos'/'Eficiencia_Energetica'/'Homecenter'/'Informe_Sodimac_5001'/'informe_final'
sys.path.append(str(project_path))

In [3]:
# import all your modules here
import json
import pandas as pd
import numpy as np

from library_ubidots_v2 import Ubidots as ubi
from library_report_v2 import Processing as pro
from library_report_v2 import Cleaning as cln

In [4]:
# to get the missing devices we must request all the devices from the account
df_devices_acc = ubi.get_available_devices_v2(label=None, level='account', page_size=1000)
df_devices = df_devices_acc[df_devices_acc['device_name'].str.startswith('HC - ')]

In [5]:
df_devices

,device_name,device_label,device_id
181,HC - Cali norte,hc---cali-norte,623c9c3ddc4fea000cf155b1
182,HC - Cali sur,hc-cali-sur,623c77baba8978a3f2083cfd
183,HC - Bucaramanga,hc---bucaramanga,623c686627da4ba193edb89a
184,HC - Palmira,hc---palmira,623b71064e9390adcdbf03fa
185,HC - Funza,hc-funza,623b27b01d0b0e9d73fd8ae7
356,HC - Calle 80,calle-80,621ccfd4c64eb2430cfc5c2c
357,HC - Cedritos,cedritos,621cca20024f3e3b10a7a8cf
358,HC - San Fernando,hc-san-fernando,6219304ea12f9d7842663540
360,HC - La Popa,hc-la-popa,6218e8fd5504633b764cecc0
389,HC - Corporativo,hc-corporativo,620fed113ab3df01dd759f93


In [6]:
DEVICE_IDS_TO_REQUEST = list(df_devices['device_id'])
df_vars = ubi.get_available_variables(DEVICE_IDS_TO_REQUEST)

In [7]:
df_vars

,variable_id,variable_label,device_id
0,636c6d727d01f6000dc0f14d,temperatura-era5,623c9c3ddc4fea000cf155b1
1,628faef6cfb70c4433267ecf,consumo-domingos-y-festivos,623c9c3ddc4fea000cf155b1
2,628faea21ecd6f65befcba6f,consumo-sabado,623c9c3ddc4fea000cf155b1
3,628fae80d245925401c4b0c5,consumo-semana,623c9c3ddc4fea000cf155b1
4,628fadf64e96714c0783d55c,consumo-promedio-por-dia,623c9c3ddc4fea000cf155b1
...,...,...,...
480,61fd7ecb52fc52000a6d6959,ac-tension-l3,61fbfb883ac637252222607e
481,61fd7eb8a28d07000a348e89,ac-tension-l2,61fbfb883ac637252222607e
482,61fd7ea3657523000ca7b5d2,ac-tension-l1,61fbfb883ac637252222607e
483,61fd21dbb458490a50a8b1b3,ea-iluminacion-patio-constructor,61fbfb883ac637252222607e


In [8]:
# paste en config
# set(df_vars.variable_label)

In [9]:
# select only energy and power variables
lst_unique_var_labels = list(set(df_vars['variable_label']))
var_labels_to_request = [s for s in lst_unique_var_labels if s.startswith(('ea-','pa-'))]
var_labels_to_request = var_labels_to_request + ['consumo-semana', 'consumo-sabado', 'consumo-domingos-y-festivos']

# exclude ea-sabados
var_labels_to_request.remove('ea-sabados')

is_whitelisted_var = df_vars['variable_label'].isin(var_labels_to_request)
VAR_IDS_TO_REQUEST = list(df_vars.loc[is_whitelisted_var, 'variable_id'])

VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [10]:
len(VAR_IDS_TO_REQUEST)

335

In [13]:
df_vars_ext = pd.merge(
    df_vars,
    df_devices
)

In [15]:
df_vars_ext.query("device_name == 'HC - Palmira'")

,variable_id,variable_label,device_id,device_name,device_label
115,626df1118da1680c71550372,temperatura,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
116,6268608aa5f2c7314cfbab59,cobertura,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
117,62672bedc2c55c000bd6d666,consumo-total-tienda,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
118,6259a53da6a098127700b787,ton-co2-acumulado,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
119,6259a4edbe96791b1917e2a3,consumo-por-area,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
120,6259a4a4adec5a1a2e4b0c0f,ea-total,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
121,6259a455be96791d8572019b,ea-iluminacion-principal,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
122,6259a2e4218f051c9f8b4738,ea-iluminacion-patio-constructor,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
123,62598e62218f0519078263b8,ea-puente-grua,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira
124,62465fb333fb228f248033d9,pa-oficinas-y-servicios,623b71064e9390adcdbf03fa,HC - Palmira,hc---palmira


In [ ]:
CHUNK_SIZE = 34
DATE_INTERVAL_REQUEST = {'start': '2022-06-01', 'end': '2022-11-01'}

df = None
lst_responses = []
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    id_request_subset = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    response = ubi.make_request(
        id_request_subset, 
        DATE_INTERVAL_REQUEST, 
    )

    lst_responses.append(response)

df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)


df['value'] = pd.to_numeric(df['value'], errors='coerce')

# df = df.astype({
#     'value':'float',
#     'variable':'category',
#     'device':'category',
#     'device_name':'category',
# })

df.to_pickle(project_path/'data'/"office_level_data_v2.pkl")